In [1]:
import pandas as pd

from src.metrics import map_score, mrr_score, ndcg_score, rmse_score
from src.models.content_based_filtering import ContentBasedFilteringRecommender
from src.utils import train_test_split, to_user_movie_matrix, make_binary_matrix

Let's load the datasets with users info, extended movies info and users' ratings for movies.

The data with movie info are extended by the information about directors, actors, runtimes, production companies, which we have took by IMDb API.

Then we split it to training/test subsets by the timestamp.

In [2]:
users = pd.read_table("../data/users.dat", sep="::", names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], engine='python')

movies = pd.read_table("../data/movies_extended.csv", sep=',', engine='python', encoding='latin1')

ratings = pd.read_table("../data/ratings.dat", sep="::", names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')
ratings['Timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')

ratings = ratings[ratings['MovieID'].isin(movies['MovieID'])]

train_ratings, test_ratings = train_test_split(ratings, 'Timestamp')
user_movie_train = to_user_movie_matrix(train_ratings)
user_movie_test = to_user_movie_matrix(test_ratings)

Now, we are going to train our content-based recommender model, which predicts ratings based on the movie features. It takes into account the similarity between the movie, we are going to predict the rating of, and the movies, which were rated by the user before.

In [3]:
model = ContentBasedFilteringRecommender()
model.train(user_movie_train, movies)

y_pred = model.predict(make_binary_matrix(user_movie_test.get_rating_matrix()), user_movie_test.get_users(), user_movie_test.get_movies())

Given the predicted ratings and test dataset, we are going to evaluate our model by four metrics:
* mean average precision (MAP)
* mean reciprocal rank (MRR)
* normalized discounted cumulative gain (NDCG)
* root mean squared error (RMSE)

In [4]:
map_score_value = map_score(user_movie_test, y_pred, top=10)
mrr_score_value = mrr_score(user_movie_test, y_pred, top=10)
ndcg_score_value = ndcg_score(user_movie_test, y_pred, top=10)
rmse_score_value = rmse_score(user_movie_test, y_pred)

print(f'Baseline MAP: {map_score_value}')
print(f'Baseline MRR: {mrr_score_value}')
print(f'Baseline NDCG: {ndcg_score_value}')
print(f'Baseline RMSE: {rmse_score_value}')

Baseline MAP: 0.16421373473072112
Baseline MRR: 0.3939339054183958
Baseline NDCG: 0.22013055941043327
Baseline RMSE: 347.6384963146933


A MAP of 0.164 indicates that, on average, about 16.4% of the top-10 recommended items are relevant.

An MRR of 0.393 means that, on average, the first relevant item appears between the 2nd and 3rd positions in the recommendation list.

NDCG measures the quality of the recommendations by considering the position of the relevant items in the list, with higher-ranked items contributing more to the score.

RMSE measures the differences between the predicted and actual ratings, which in our case is quite big, but significantly lower than baseline, and just a little bit lower than content-based filtering using only genres info.

So, adding more info about movie in our case improved the results, but not singnificanlty.